In [ ]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00
   ━━━

In [ ]:
!git clone https://github.com/bisariyon/CHSW-Trash_Identification.git

Cloning into 'CHSW-Trash_Identification'...
remote: Enumerating objects: 2646, done.
remote: Counting objects: 100% (1470/1470), done.
remote: Compressing objects: 100% (845/845), done.
remote: Total 2646 (delta 611), reused 1464 (delta 609), pack-reused 1176 (from 1)
Receiving objects: 100% (2646/2646), 272.74 MiB | 35.61 MiB/s, done.
Resolving deltas: 100% (619/619), done.
Updating files: 100% (2503/2503), done.


In [ ]:
train_dataset_path = "./CHSW-Trash_Identification/train_dataset"
test_dataset_path = "./CHSW-Trash_Identification/test_dataset"

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import edgeimpulse as ei

import os
from PIL import Image

In [ ]:

# Function to verify images and remove corrupted files
def verify_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify if the image can be opened
            except (IOError, SyntaxError) as e:
                print(f"Removing corrupted or unsupported file: {file_path}")
                os.remove(file_path)

In [ ]:
# Verify images in the dataset
verify_images(train_dataset_path)
verify_images(test_dataset_path)

Removing corrupted or unsupported file: ./CHSW-Trash_Identification/train_dataset/Paper/53.jpg
Removing corrupted or unsupported file: ./CHSW-Trash_Identification/train_dataset/Paper/86.jpg


In [ ]:
# Set the Edge Impulse API key
ei.API_KEY = "ei_f560eb38285fc0e05360ea854f9cb1c6eddb7b76250ebd319462957951bb296a"  # Replace with your Edge Impulse API key


In [ ]:
# Labels for the dataset
labels = ["Plastic", "Metal Can", "Paper", "Rotten Fruits", "E-waste", "Water Bottle"]
num_classes = len(labels)

In [ ]:
# Filename for the exported model
deploy_filename = "my_model_cpp.zip"

In [ ]:
# Data preprocessing with ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    validation_split=0.2  # Split dataset into 80% training and 20% validation
)

In [ ]:
# Load training and validation datasets
image_size = (64, 64)  # Resize images to 64x64
batch_size = 32

train_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 743 images belonging to 6 classes.
Found 184 images belonging to 6 classes.


In [ ]:
# Testing dataset (Assumes a separate test dataset directory is provided)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Only rescaling for testing
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,  # Path to the test dataset directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False  # No shuffling to maintain consistent batch order
)

Found 230 images belonging to 6 classes.


In [ ]:
# Define the input shape
input_shape = (*image_size, 3)  # 3 channels for RGB

In [ ]:
# Build the model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(num_classes, activation="softmax")
])

In [ ]:
# Compile the model
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
10/24 [===========>..................] - ETA: 6s - loss: 1.8782 - accuracy: 0.1729

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


24/24 [==============================] - 14s 527ms/step - loss: 1.6844 - accuracy: 0.3230 - val_loss: 1.4531 - val_accuracy: 0.5761
Epoch 2/10
24/24 [==============================] - 12s 510ms/step - loss: 1.0169 - accuracy: 0.6231 - val_loss: 0.7550 - val_accuracy: 0.7337
Epoch 3/10
24/24 [==============================] - 12s 488ms/step - loss: 0.6999 - accuracy: 0.7510 - val_loss: 0.6058 - val_accuracy: 0.8207
Epoch 4/10
24/24 [==============================] - 13s 520ms/step - loss: 0.5718 - accuracy: 0.8116 - val_loss: 0.5369 - val_accuracy: 0.8315
Epoch 5/10
24/24 [==============================] - 12s 501ms/step - loss: 0.4338 - accuracy: 0.8694 - val_loss: 0.6819 - val_accuracy: 0.7554
Epoch 6/10
24/24 [==============================] - 13s 522ms/step - loss: 0.3590 - accuracy: 0.8802 - val_loss: 0.6271 - val_accuracy: 0.8098
Epoch 7/10
24/24 [==============================] - 13s 528ms/step - loss: 0.2226 - accuracy: 0.9381 - val_loss: 0.5605 - val_accuracy: 0.8804
Epoch 8/10

In [ ]:
# Evaluate the model on the validation set
score = model.evaluate(validation_generator, verbose=0)
print(f"Validation loss: {score[0]}")
print(f"Validation accuracy: {score[1]}")


Validation loss: 0.6455987691879272
Validation accuracy: 0.8586956262588501


In [ ]:
# Evaluate model on test set
score = model.evaluate(test_generator, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")

Test loss: 1.089106798171997
Test accuracy: 0.747826099395752


In [ ]:
# List the available profile target devices
ei.model.list_profile_devices()

['alif-he',
 'alif-hp',
 'ambiq-apollo4',
 'arduino-nano-33-ble',
 'arduino-nicla-vision',
 'arduino-nicla-vision-m4',
 'portenta-h7',
 'brainchip-akd1000',
 'brickml',
 'cortex-m4f-80mhz',
 'cortex-m7-216mhz',
 'nxp-imx93-npu',
 'nxp-imx93-cpu',
 'espressif-esp32',
 'himax-we-i',
 'himax-wiseeye2',
 'himax-wiseeye2-ethos',
 'infineon-cy8ckit-062s2',
 'infineon-cy8ckit-062-ble',
 'mbp-16-2020',
 'memryx-mx3',
 'microchip-sama7g54',
 'nordic-nrf52840-dk',
 'nordic-nrf5340-dk',
 'nordic-nrf9151-dk',
 'nordic-nrf9160-dk',
 'nordic-nrf9161-dk',
 'jetson-nano',
 'jetson-orin-nx',
 'jetson-orin-nano',
 'openmv-h7p',
 'particle-boron',
 'particle-p2',
 'raspberry-pi-4',
 'raspberry-pi-5',
 'raspberry-pi-rp2040',
 'renesas-ck-ra6m5',
 'renesas-ek-ra8d1',
 'renesas-rzg2l',
 'renesas-rzv2h-cpu',
 'renesas-rzv2h',
 'renesas-rzv2l-cpu',
 'renesas-rzv2l',
 'st-iot-discovery-kit',
 'st-stm32n6',
 'seeed-sense-cap',
 'wio-terminal',
 'seeed-vision-ai',
 'silabs-xg24',
 'silabs-thunderboard-sense-2',


In [ ]:
# Profile your model for target hardware
try:
    profile = ei.model.profile(model=model, device="arduino-nano-33-ble")
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

Target results for float32:
{
    "device": "arduino-nano-33-ble",
    "tfliteFileSizeBytes": 6507112,
    "isSupportedOnMcu": true,
    "memory": {
        "tflite": {
            "ram": 742198,
            "rom": 6554184,
            "arenaSize": 741894
        },
        "eon": {
            "ram": 616880,
            "rom": 6526704,
            "arenaSize": 615520
        },
        "eonRamOptimized": {
            "ram": 259760,
            "rom": 6536440,
            "arenaSize": 257088
        }
    },
    "timePerInferenceMs": 26861
}


Performance on device types:
{
    "variant": "float32",
    "lowEndMcu": {
        "description": "Estimate for a Cortex-M0+ or similar, running at 40MHz",
        "timePerInferenceMs": 67275,
        "memory": {
            "tflite": {
                "ram": 742044,
                "rom": 6542408
            },
            "eon": {
                "ram": 616784,
                "rom": 6523456
            },
            "eonRamOptimized": {
   

In [ ]:
# List available deployment targets
ei.model.list_deployment_targets()

['syntiant-ndp101-lib',
 'syntiant-ndp120-lib',
 'syntiant-ndp120-lib-tdk-v14',
 'syntiant-ndp120-lib-ndp-v1-15-0',
 'syntiant-ndp101',
 'syntiant-nicla-ndp120',
 'syntiant-avnet-rasyn']

In [ ]:
# Set model information, such as your list of labels
model_output_type = ei.model.output_type.Classification(labels=labels)


In [ ]:
# Set model input type
model_input_type = ei.model.input_type.ImageInput()


In [ ]:

# Deploy the model as a C++ library
deploy_bytes = None
try:
    deploy_bytes = ei.model.deploy(
        model=model,
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target="zip"
    )
except Exception as e:
    print(f"Could not deploy: {e}")

In [ ]:
# Save the deployed model as a .zip file
if deploy_bytes:
    with open(deploy_filename, "wb") as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model deployed successfully! Saved as {deploy_filename}.")

Model deployed successfully! Saved as my_model_cpp.zip.
